load_keops on terminal 

In [1]:
import os.path
import sys
sys.path.append('/home/user/Code/keops/keopscore')
sys.path.append('/home/user/Code/keops/pykeops')
import os
import pickle
import pandas as pd
import tracemalloc
import pickle
import shutil
import time
import cv2

In [2]:
import pykeops
pykeops.test_torch_bindings() 

pyKeOps with torch bindings is working!


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import time

import torch
from geomloss import SamplesLoss

use_cuda = torch.cuda.is_available()
dtype = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor

In [4]:
use_cuda

True

In [5]:
import numpy as np
import torch
from random import choices
import imageio
from matplotlib import pyplot as plt


def load_image(fname):
    img = imageio.imread(fname)  # RGB
    return img / 255.0  # Normalized to [0,1]


def RGB_cloud(fname, sampling, dtype=torch.FloatTensor):
    A = load_image(fname)
    A = A[::sampling, ::sampling, :]
    return torch.from_numpy(A).type(dtype).view(-1, 3)


def display_cloud(ax, x):
    x_ = x.detach().cpu().numpy()
    ax.scatter(x_[:, 0], x_[:, 1], x_[:, 2], s=25 * 500 / len(x_), c=x_)


def display_image(ax, x):
    W = int(np.sqrt(len(x)))
    x_ = x.view(W, W, 3).detach().cpu().numpy()
    ax.imshow(x_)


In [9]:
import os, numpy, PIL
from PIL import Image
PATH = '/media/user/Watson/GBM_LGG/patches_select/'
PATH_GeomLoss = '/media/user/Watson/GBM_LGG/Normalisation/'

In [10]:
datasets = os.listdir(PATH)
len(datasets)

110

# Image ref (= mean of pixels)

In [1]:
for dataset in datasets :
    print(dataset)
    if not os.path.exists('/home/user/Code/Etude_normalisation/GBM_LGG/GemLoss_average_images/'+dataset+"_Average.png") :
        # Access all PNG files in directory
        allfiles=os.listdir(PATH+dataset)
        print(len(allfiles))
        imlist=[filename for filename in allfiles if  filename[-4:] in [".jpg"]]

        # Assuming all images are the same size, get dimensions of first image
        w,h=Image.open(PATH+dataset+'/'+imlist[0]).size
        N=len(imlist)

        # Create a numpy array of floats to store the average (assume RGB images)
        arr=numpy.zeros((h,w,3),np.float64)

        # Build up average pixel intensities, casting each image as an array of floats
        for im in imlist:
            imarr=numpy.array(Image.open(PATH+dataset+'/'+im),dtype=np.float64)
            try :
                arr=arr+imarr/N
            except ValueError :
                pass

        # Round values in array and cast as 8-bit integer
        arr=numpy.array(numpy.round(arr),dtype=numpy.uint8)

        # Generate, save and preview final image
        out=Image.fromarray(arr,mode="RGB")
        out.save('/home/user/Code/Etude_normalisation/GBM_LGG/GemLoss_average_images/'+dataset+"_Average.png")
        out.show()
    else:
        print('already exists')

In [12]:
liste_average = []
for i in os.listdir('/home/user/Code/Etude_normalisation/GBM_LGG/GemLoss_average_images'):
    if i[-11:]=='Average.png' :
        liste_average.append(i)
len(liste_average)

111

In [2]:
for im in liste_average:
    imarr=numpy.array(Image.open('/home/user/Code/Etude_normalisation/GBM_LGG/GemLoss_average_images/'+im),dtype=np.float64)
    arr=arr+imarr/N
arr=numpy.array(numpy.round(arr),dtype=numpy.uint8)

# Generate, save and preview final image
out=Image.fromarray(arr,mode="RGB")
out.save("/home/user/Code/Etude_normalisation/GBM_LGG/GemLoss_average_images/TCGA_LGG_GBM_Average.png")
out.show()

# GeomLoss

In [17]:
def color_transfer(X, Y, loss, lr=1):
    """Flows along the gradient of the loss function.

    Parameters:
        loss ((x_i,y_j) -> torch float number):
            Real-valued loss function.
        lr (float, default = 1):
            Learning rate, i.e. time step.
    """

    # Parameters for the gradient descent
    Nsteps = 11
    display_its = [1, 10]

    # Make sure that we won't modify the reference samples
    x_i, y_j = X.clone(), Y.clone()

    # We're going to perform gradient descent on Loss(α, β)
    # wrt. the positions x_i of the diracs masses that make up α:
    x_i.requires_grad = True

    t_0 = time.time()

    # plt.figure(figsize=(12, 12))
    # k = 3
    # ax = plt.subplot(2, 2, 1)
    # display_image(ax, x_i)
    # ax.set_title("Source image")
    # plt.xticks([], [])
    # plt.yticks([], [])
    # ax = plt.subplot(2, 2, 2)
    # display_image(ax, y_j)
    # ax.set_title("Target image")
    # plt.xticks([], [])
    # plt.yticks([], [])

    for i in range(Nsteps):  # Euler scheme ===============
        # Compute cost and gradient
        L_αβ = loss(x_i, y_j)
        [g] = torch.autograd.grad(L_αβ, [x_i])

        # if i in display_its:  # display
        #     ax = plt.subplot(2, 2, k)
        #     display_image(ax, x_i)
        #     ax.set_title("it = {}".format(i))
        #     k = k + 1
        #     plt.xticks([], [])
        #     plt.yticks([], [])

        # in-place modification of the tensor's values
        x_i.data -= lr * len(x_i) * g
    return x_i
    # plt.title(
    #     "it = {}, elapsed time: {:.2f}s/it".format(i, (time.time() - t_0) / Nsteps)
    # )
    # plt.tight_layout()

In [18]:
methods = ['GeomLoss_energy', 'GeomLoss_sinkhorn', 'GeomLoss_laplacian', 'GeomLoss_gaussian']

In [3]:
organs = ['Adrenal_gland', 'Bile-duct', 'Bladder', 'Breast', 'Cervix', 'Colon', 'Esophagus', 'HeadNeck', 'Kidney', 'Liver', 'Lung', 'Ovarian', 'Pancreatic', 'Prostate', 'Skin', 'Stomach', 'Testis', 'Thyroid', 'Uterus']

In [21]:
df = pd.DataFrame()

In [4]:
PATH =  "/media/user/Watson/PanNuke/normalisation_study/original/patches/"/"


for method in [ 'GeomLoss_sinkhorn_target','GeomLoss_sinkhorn_gradient', 'GeomLoss_sinkhorn_Average']: 
    print('-----', method, '----------')
    if not os.path.exists("/media/user/Watson/PanNuke/normalisation/"+method) :
        os.mkdir("/media/user/Watson/PanNuke/normalisation/"+method)
    PATH_method =  "/media/user/Watson/PanNuke/normalisation/"+method+"/patches/"
    if not os.path.exists(PATH_method) :
        os.mkdir(PATH_method)
    for organ in organs :

        if not os.path.exists(PATH_method+organ) :
            os.mkdir(PATH_method+organ)
        for i in os.listdir(PATH+organ):
            X = RGB_cloud(PATH +organ+'/'+i, sampling, dtype)
            if method == 'GeomLoss_sinkhorn_target' :
                Y = RGB_cloud("/home/user/Code/Etude_normalisation/GBM_LGG/image_target.png", sampling, dtype)
                x_i = color_transfer(X, Y, SamplesLoss('sinkhorn', blur=0.9, reach=0.4))
            elif method == 'GeomLoss_sinkhorn_gradient' :
                Y = RGB_cloud("/home/user/Code/Etude_normalisation/PanNuke/Gradient_PanNuke/PanNuke_gradient.png", sampling, dtype)
                x_i = color_transfer(X, Y, SamplesLoss('sinkhorn', blur=0.9, reach=0.4))
            elif method == 'GeomLoss_sinkhorn_Average' :
                Y = RGB_cloud("/home/user/Code/Etude_normalisation/PanNuke/Average_PanNuke/PanNuke_Average.png", sampling, dtype)
                x_i = color_transfer(X, Y, SamplesLoss('sinkhorn', blur=0.9, reach=0.4))
            W = int(np.sqrt(len(x_i)))
            try : 
                x_ = x_i.view(W, W, 3).detach().cpu().numpy()
            except RuntimeError:
                print('ok')
                continue
            x_ = x_[:,:,::-1]*255
            cv2.imwrite(PATH_method+organ+"/"+i, x_)

    

